In [1]:
import numpy as np
import pandas as pd
import requests  

Downloading protein's sequences from G4IPDB http://people.iiti.ac.in/~amitk/bsbe/ipdb/index.php

In [2]:
!rm -r G4IPDB
!mkdir G4IPDB
!mkdir G4IPDB/DNA
!mkdir G4IPDB/RNA

rm: cannot remove 'G4IPDB': No such file or directory


In [91]:
!ls G4IPDB

DNA  RNA


In [ ]:
nans = 0
for i in range(0, 27):
    url = 'http://people.iiti.ac.in/~amitk/bsbe/ipdb/g4dna.php?start=' + str(i)
    table = pd.read_html(url)[0]
    for id, prot_id in zip(table['Interaction ID'], table['UniProt-ID']):
      if prot_id is not np.NaN:
          url = "https://rest.uniprot.org/uniprotkb/" + prot_id + ".fasta"
          r = requests.get(url)
          
          with open('/content/PDBP-Fusion/G4IPDB/DNA/' +id +'_'+ prot_id + '.txt', 'w') as f: 
            f.write(str(r.content).split("\\n")[0][2:] + '\n')
            f.write("".join(str(r.content).split("\\n")[1:-1]) + '\n')

      else:
        nans+=1

print("There are ", nans, "interactions without UniProt-ID")


In [93]:
!cat /content/PDBP-Fusion/G4IPDB/DNA/* >> /content/PDBP-Fusion/G4IPDB/DNA_prot.txt
!grep -v ">" /content/PDBP-Fusion/G4IPDB/DNA_prot.txt > /content/PDBP-Fusion/G4IPDB/DNA_prot_PDBP.txt

In [96]:
!wc -l /content/PDBP-Fusion/G4IPDB/DNA_prot.txt

254 /content/PDBP-Fusion/G4IPDB/DNA_prot.txt


In [97]:
for i in range(0, 16):
    url = 'http://people.iiti.ac.in/~amitk/bsbe/ipdb/g4rna.php?start=' + str(i)
    table = pd.read_html(url)[0].dropna(subset=['UniProt-ID'])
    
    for id, prot_id in zip(table['Interaction ID'], table['UniProt-ID']):
      #print(id, prot_id)
      if prot_id and prot_id != 'nan' and prot_id is not np.NaN:
          url = "https://rest.uniprot.org/uniprotkb/" + prot_id + ".fasta"
          r = requests.get(url)
          
          with open('/content/PDBP-Fusion/G4IPDB/RNA/' +id +'_'+ prot_id + '.txt', 'w') as f: 
            f.write(str(r.content).split("\\n")[0][2:] + '\n')
            f.write("".join(str(r.content).split("\\n")[1:-1])+ '\n')
   

In [98]:
!head /content/PDBP-Fusion/G4IPDB/DNA_prot_PDBP.txt

MSGWESYYKTEGEEEEEEEESPDPGGEYKYSGRDSLIFLVDASRAMFDSQGEDEITPFDMSIQCIQSVYTSKIISSNRDLLGVVFYGTEKDKNSVNFKNIYVLQELDNPGAKRVLELDQFKGQQGKKHFQDTIGHGSDYSLSEVLWVCANLFSDVQVKMSHKRIMLFTNEDDPHGNDSAKASRARTKANDLRDTGIFLDLMHLKRRGGFDISLFYRDIMSIAEDEDLGVHFEESSKLEDLLRKVRAKETKKRVLSRLRFKLGKDVALMVGIYNLIQKANKPFPVRLYRETNEPVKTKTRTFNVNTGSLLLPSDTKRSQTYGSRQIVLEKEETEELKRFDEPGLILMGFKALVMLKKHHYLRPSLFVYPEESLVNGSSTLFSALLTKCLEKEVMAVCRYTSRKNVPPYFVALVPQEEELDDQNIQVTPAGFQLVFLPYADDKRKVPFTEKVMANPEQIDKMKAIVHNVRFTYRSDSFENPVLQQHFRNLEALALDMMESEQVVDLTLPKAEAIKKRLGSLADEFKELVYPPGYNPEGKATKRKQDDEGSASKKPKEELSEEELKAHFAKGTLGKLTVPTLKEVCKAYGLKSGPKKQELLDALTRHFQKN
MAWSANKAAVVLCMDVGVAMGNSFPGEESSFEQAKKVMTMFVQRQVFSESKDEIALVLFGTDNTNNALASEDQYQNITVHRHLMLPDFDLLEDIESKIQLGSRQADILDALIVCMDLIQRETIGKKFEKKHIEVFTDLSSPFSQDQLDVIICNLKKSGISLQFFLPFPISKNDETGDRGDGDLGLDHCGPSFPQKGITEQQKEGICMVERVMVSLEGEDGLDEIYSFSESLRRLCVFKKIERRSMPWSCQLTIGPDLSIKIVAYKSIVQEKVKKSWIVVDARTLKKEDIRKETVYCLNDDDETEVSKEDTIQGFRYGSDIIPFSKVDEEQMKYKSEGKCFSVLGFCRSSQVHRRFFMGYQVLKVFAAKDDEAAAVALSSLIHALDELNMVA

Apllying PDBP-Fusion

In [5]:
!git clone https://github.com/hfuulgb/PDBP-Fusion.git
%cd PDBP-Fusion/

Cloning into 'PDBP-Fusion'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 53 (delta 14), reused 36 (delta 10), pack-reused 0
Unpacking objects: 100% (53/53), 12.55 MiB | 9.93 MiB/s, done.
/content/PDBP-Fusion


Transform data for PDBP

In [99]:
!cat /content/PDBP-Fusion/G4IPDB/RNA/* >> /content/PDBP-Fusion/G4IPDB/RNA_prot.txt
!grep -v ">" /content/PDBP-Fusion/G4IPDB/RNA_prot.txt > /content/PDBP-Fusion/G4IPDB/RNA_prot_PDBP.txt

In [100]:
#edited predict.py

from keras.models import load_model
import numpy as np
from datetime import datetime
import tensorflow as tf
import numpy as np
import pdb

def get_seq_concolutional_array(seq): #from encode_schema.py
    # seq = seq.replace('U', 'T')
    alpha = 'ACDEFGHIKLMNPQRSTVWY'
    row = (len(seq))
    new_array = np.zeros((row, 20))

    for i, val in enumerate(seq):

        if val not in 'ACDEFGHIKLMNPQRSTVWY':
            if val == 'Z':
                new_array[i] = np.array([0.0] * 20)
            # if val == 'S':
            #     new_array[i] = np.array([0, 0.5, 0.5, 0, 0])
            continue

        try:
            index = alpha.index(val)
            new_array[i][index] = 1
        except ValueError:
            pdb.set_trace()
    return new_array

def read_seq_onehot(seq_file, str_len=600): #from encode_schema.py fix const 600 (was 850)
    seq_list = []
    seq = ''
    with open(seq_file, 'r') as fp:
        index = 0
        for line in fp:
            index += 1
            string = line
            if len(line)>=str_len:
                string=string[:str_len]
            else:
                string=string+'Z'*(str_len-len(string))
            
            assert len(string)==str_len
            line = string

            seq = line
            seq_array = get_seq_concolutional_array(seq)
            seq_list.append(seq_array)
        print(index)
    return np.array(seq_list, dtype=object)


class Pred():
    def __init__(self,model_path='./model_save/model_'):
        self.item_model_path = '/content/PDBP-Fusion/model_save/model_0.hdf5'
        #self.model = load_model(item_model_path)

    def sample_predict1(self,string,str_len=600):
        # make sure the input sequnce less than 850, otherwise drop string when length excell 850
        if len(string)>=str_len:
            string=string[:str_len]
        else:
            string=string+'Z'*(str_len-len(string))
        #print('len: ',str_len)
        assert len(string)==str_len

        #向量化
        #print(get_seq_concolutional_array(string)[:5,:])
        string_vector=get_seq_concolutional_array(string)
        string_vector=np.expand_dims(string_vector,0)

        #load model and predict
        result=[]
        self.model = load_model(self.item_model_path)
        predict=self.model.predict(string_vector)[0]
        print('prediction is {}'.format(predict))
        result.append(predict)
        print(result)

        return result

    def sample_predict(self,string,str_len=600):
        # make sure the input sequnce less than 850, otherwise drop string when length excell 850

        seq = read_seq_onehot(string)
        print(seq.shape)
        # seq = np.asarray(seq).astype(np.float32)
        # string_vector=get_seq_concolutional_array(string)
        # string_vector=np.expand_dims(string_vector,0)

        #load model and predict
        result=[]
        self.model = load_model(self.item_model_path)
        predict=self.model.predict(tf.convert_to_tensor(seq, dtype=tf.float32))
        print('prediction is {}'.format(predict))
        result.append(predict)
        print(result)

        return result

Pr=Pred()
start = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

ans = Pr.sample_predict(string='/content/PDBP-Fusion/G4IPDB/DNA_prot_PDBP.txt')
print('')
end = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
print('start: %s' % start)
print('end: %s' % end)

classes = np.argmax(ans[0], axis=1)
print('DBPs = ', np.sum(classes), ' nonDBPs = ', len(classes) - np.sum(classes))

127
(127, 600, 20)
4/4 [==============================] - 1s 57ms/step
prediction is [[5.83679736e-01 4.29693490e-01]
 [8.40219498e-01 1.67507127e-01]
 [9.78367031e-01 3.24326120e-02]
 [7.32459575e-02 9.22092736e-01]
 [9.96866286e-01 5.64933149e-03]
 [3.49269696e-02 9.62394714e-01]
 [6.34717941e-01 3.81487906e-01]
 [3.25407743e-01 6.64800465e-01]
 [6.78974271e-01 3.29970837e-01]
 [6.78974271e-01 3.29970837e-01]
 [1.09451793e-01 8.82217526e-01]
 [7.16405153e-01 3.02873582e-01]
 [2.07491249e-01 7.95378745e-01]
 [7.45811760e-01 2.52812445e-01]
 [4.90831405e-01 5.27980804e-01]
 [3.78930658e-01 6.21086776e-01]
 [4.01239723e-01 5.99572659e-01]
 [9.05213535e-01 1.05144702e-01]
 [2.48763144e-01 7.35537827e-01]
 [4.07715976e-01 6.03054225e-01]
 [6.32884502e-02 9.29804385e-01]
 [6.32884502e-02 9.29804385e-01]
 [6.32884502e-02 9.29804385e-01]
 [7.32459575e-02 9.22092736e-01]
 [7.32459575e-02 9.22092736e-01]
 [7.32459575e-02 9.22092736e-01]
 [8.57461840e-02 9.05430436e-01]
 [1.41407967e-01 8.71019

In [101]:
Pr=Pred()
start = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

ans = Pr.sample_predict(string='/content/PDBP-Fusion/G4IPDB/RNA_prot_PDBP.txt')
print('')
end = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
print('start: %s' % start)
print('end: %s' % end)

classes = np.argmax(ans[0], axis=1)
print('DBPs = ', np.sum(classes), ' nonDBPs = ', len(classes) - np.sum(classes))

66
(66, 600, 20)
3/3 [==============================] - 1s 34ms/step
prediction is [[0.07324596 0.92209274]
 [0.773034   0.28438127]
 [0.773034   0.28438127]
 [0.773034   0.28438127]
 [0.773034   0.28438127]
 [0.773034   0.28438127]
 [0.773034   0.28438127]
 [0.71867406 0.29779026]
 [0.51788986 0.4992013 ]
 [0.5696449  0.45350593]
 [0.9942555  0.00794362]
 [0.99920696 0.00171413]
 [0.773034   0.28438127]
 [0.773034   0.28438127]
 [0.68378276 0.3122897 ]
 [0.31216824 0.69682634]
 [0.13436271 0.8701434 ]
 [0.8282022  0.18231778]
 [0.68378276 0.3122897 ]
 [0.25730535 0.73407227]
 [0.31216824 0.69682634]
 [0.13436271 0.8701434 ]
 [0.8282022  0.18231778]
 [0.37893066 0.6210868 ]
 [0.755363   0.25439093]
 [0.0646235  0.93154573]
 [0.9950545  0.00854357]
 [0.92256933 0.10080437]
 [0.9935457  0.0079199 ]
 [0.37893066 0.6210868 ]
 [0.773034   0.28438124]
 [0.37893066 0.6210867 ]
 [0.19458587 0.81126785]
 [0.19458587 0.81126785]
 [0.37893066 0.6210868 ]
 [0.773034   0.28438127]
 [0.37893066 0.62

Apllying DBPred

In [103]:
! rm -r dbpred

In [104]:
%cd /content

/content


In [105]:
! wget https://webs.iiitd.edu.in/raghava/dbpred/dbpred.zip
!unzip dbpred.zip
%cd /content/dbpred

--2023-05-09 19:05:18--  https://webs.iiitd.edu.in/raghava/dbpred/dbpred.zip
Resolving webs.iiitd.edu.in (webs.iiitd.edu.in)... 103.25.231.42
Connecting to webs.iiitd.edu.in (webs.iiitd.edu.in)|103.25.231.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9379197 (8.9M) [application/zip]
Saving to: ‘dbpred.zip’

dbpred.zip          100%[===================>]   8.94M  3.33MB/s    in 2.7s    

2023-05-09 19:05:22 (3.33 MB/s) - ‘dbpred.zip’ saved [9379197/9379197]

Archive:  dbpred.zip
   creating: dbpred/
  inflating: dbpred/example_output.txt  
  inflating: dbpred/envfile          
  inflating: dbpred/LICENSE          
  inflating: dbpred/dbpred.py        
  inflating: dbpred/INSTALLATION     
  inflating: dbpred/README.md        
   creating: dbpred/prog/
  inflating: dbpred/prog/model_PCB.h5  
  inflating: dbpred/prog/model.h5    
  inflating: dbpred/prog/model_PSSM.h5  
  inflating: dbpred/prog/model_AAB.h5  
  inflating: dbpred/example_seq.fa   
/content/db

In [106]:
!head /content/PDBP-Fusion/G4IPDB/DNA_prot.txt

>tr|Q9QVN9|Q9QVN9_CRIGR KU70=DNA-dependent protein kinase 70 kDa DNA-binding subunit/DNA-PK Ku70 subunit OS=Cricetulus griseus OX=10029 PE=3 SV=1
MSGWESYYKTEGEEEEEEEESPDPGGEYKYSGRDSLIFLVDASRAMFDSQGEDEITPFDMSIQCIQSVYTSKIISSNRDLLGVVFYGTEKDKNSVNFKNIYVLQELDNPGAKRVLELDQFKGQQGKKHFQDTIGHGSDYSLSEVLWVCANLFSDVQVKMSHKRIMLFTNEDDPHGNDSAKASRARTKANDLRDTGIFLDLMHLKRRGGFDISLFYRDIMSIAEDEDLGVHFEESSKLEDLLRKVRAKETKKRVLSRLRFKLGKDVALMVGIYNLIQKANKPFPVRLYRETNEPVKTKTRTFNVNTGSLLLPSDTKRSQTYGSRQIVLEKEETEELKRFDEPGLILMGFKALVMLKKHHYLRPSLFVYPEESLVNGSSTLFSALLTKCLEKEVMAVCRYTSRKNVPPYFVALVPQEEELDDQNIQVTPAGFQLVFLPYADDKRKVPFTEKVMANPEQIDKMKAIVHNVRFTYRSDSFENPVLQQHFRNLEALALDMMESEQVVDLTLPKAEAIKKRLGSLADEFKELVYPPGYNPEGKATKRKQDDEGSASKKPKEELSEEELKAHFAKGTLGKLTVPTLKEVCKAYGLKSGPKKQELLDALTRHFQKN
>tr|Q60449|Q60449_CRIGR X-ray repair cross-complementing protein 5 OS=Cricetulus griseus OX=10029 GN=Ku86 PE=2 SV=1
MAWSANKAAVVLCMDVGVAMGNSFPGEESSFEQAKKVMTMFVQRQVFSESKDEIALVLFGTDNTNNALASEDQYQNITVHRHLMLPDFDLLEDIESKIQLGSRQADILDALIVCMDLIQRETIGKKFEK

In [107]:
!python dbpred.py -i /content/PDBP-Fusion/G4IPDB/DNA_prot.txt -o G4IPDB_DNA.csv

2023-05-09 19:10:17.595835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
# This program DB0Pred is developed for predicting DNA-interacting resdiues  #
# in protein sequence, developed by Prof G. P. S. Raghava group. #
# ############################################################################
Summary of Parameters:
Input File:  /content/PDBP-Fusion/G4IPDB/DNA_prot.txt ; Threshold:  0.05 ; Job Type:  1
Output File:  G4IPDB_DNA.csv
==== Initiating the process of Prediction using amino acid binaray profiles as input features: please wait ...


In [119]:
!head /content/dbpred/G4IPDB_DNA.csv

>tr|Q9QVN9|Q9QVN9_CRIGR
MSGWESYYKTEGEEEEEEEESPDPGGEYKYSGRDSLIFLVDASRAMFDSQGEDEITPFDMSIQCIQSVYTSKIISSNRDLLGVVFYGTEKDKNSVNFKNIYVLQELDNPGAKRVLELDQFKGQQGKKHFQDTIGHGSDYSLSEVLWVCANLFSDVQVKMSHKRIMLFTNEDDPHGNDSAKASRARTKANDLRDTGIFLDLMHLKRRGGFDISLFYRDIMSIAEDEDLGVHFEESSKLEDLLRKVRAKETKKRVLSRLRFKLGKDVALMVGIYNLIQKANKPFPVRLYRETNEPVKTKTRTFNVNTGSLLLPSDTKRSQTYGSRQIVLEKEETEELKRFDEPGLILMGFKALVMLKKHHYLRPSLFVYPEESLVNGSSTLFSALLTKCLEKEVMAVCRYTSRKNVPPYFVALVPQEEELDDQNIQVTPAGFQLVFLPYADDKRKVPFTEKVMANPEQIDKMKAIVHNVRFTYRSDSFENPVLQQHFRNLEALALDMMESEQVVDLTLPKAEAIKKRLGSLADEFKELVYPPGYNPEGKATKRKQDDEGSASKKPKEELSEEELKAHFAKGTLGKLTVPTLKEVCKAYGLKSGPKKQELLDALTRHFQKN
-++++++---------------------++-+++---------+---------------------+--+++++++++++-----++-++++++++++++--------+---++-------++++++++------+--++--------------+-+-++++++---------------++-++++++++--++---+++--+++++++++++++++-------------------+-----+++++++++++++++++++++++--++++-++--++-+++++-+-++--+---++++++++++----------++++++++++---------------------++++++++++++++-+------

In [112]:
!python dbpred.py -i /content/PDBP-Fusion/G4IPDB/RNA_prot.txt -o G4IPDB_RNA.csv

2023-05-09 19:13:10.458368: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
# This program DB0Pred is developed for predicting DNA-interacting resdiues  #
# in protein sequence, developed by Prof G. P. S. Raghava group. #
# ############################################################################
Summary of Parameters:
Input File:  /content/PDBP-Fusion/G4IPDB/RNA_prot.txt ; Threshold:  0.05 ; Job Type:  1
Output File:  G4IPDB_RNA.csv
==== Initiating the process of Prediction using amino acid binaray profiles as input features: please wait ...


In [118]:
!head /content/dbpred/G4IPDB_RNA.csv

>sp|Q15554|TERF2_HUMAN
MAAGAGTAGPASGPGVVRDPAASQPRKRPGREGGEGARRSDTMAGGGGSSDGSGRAAGRRASRSSGRARRGRHEPGLGGPAERGAGEARLEEAVNRWVLKFYFHEALRAFRGSRYGDFRQIRDIMQALLVRPLGKEHTVSRLLRVMQCLSRIEEGENLDCSFDMEAELTPLESAINVLEMIKTEFTLTEAVVESSRKLVKEAAVIICIKNKEFEKASKILKKHMSKDPTTQKLRNDLLNIIREKNLAHPVIQNFSYETFQQKMLRFLESHLDDAEPYLLTMAKKALKSESAASSTGKEDKQPAPGPVEKPPREPARQLRNPPTTIGMMTLKAAFKTLSGAQDSEAAFAKLDQKDLVLPTQALPASPALKNKRPRKDENESSAPADGEGGSELQPKNKRMTISRLVLEEDSQSTEPSAGLNSSQEAASAPPSKPTVLNQPLPGEKNPKVPKGKWNSSNGVEEKETWVEEDELFQVQAAPDEDSTTNITKKQKWTVEESEWVKAGVQKYGEGNWAAISKNYPFVNRTAVMIKDRWRTMKRLGMN
----------------------++-+++--++++++++++------+-+++++++++++++++++++++++++--+------+-----+-----+++-++++++++++++++++++++++-++-++--------++++-++--+--+----------------------------------++------------++++++---+-+++++++++-+++-+++-++-+++++-++---+--++++--+------++++-+++---------------------+---------------------------+----------++-++++-++--+-----+---------------------------++++-+------------------+-++++++-++-------------------------------

In [120]:
!cat /content/dbpred/dbpred.py

##############################################################################
# DBPred is developed for predicting the DNA-interacting residues in protein  #
# using primary structure. It is developed by Prof G. P. S. Raghava's group.  #
# Please cite: DBPred						      #
# ############################################################################
import os
import pickle
import numpy as np
import pandas as pd
import sys
import csv
import re
import glob
import time
from time import sleep
#from tqdm import tqdm
import argparse
import warnings
from argparse import RawTextHelpFormatter
import tensorflow as tf
from keras.models import load_model
import uuid
warnings.filterwarnings('ignore')


parser = argparse.ArgumentParser(description='Please provide following arguments',formatter_class=RawTextHelpFormatter)
## Read Arguments from command
parser.add_argument("-i", "--input", type=str, required=True, help="Input: Protein sequences in FASTA format")
parser.add_argument("-o", "--output",t

In [1]:
!wget https://github.com/angusev/coursework_template/archive/refs/heads/master.zip

--2023-05-10 17:34:23--  https://github.com/angusev/coursework_template/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/angusev/coursework_template/zip/refs/heads/master [following]
--2023-05-10 17:34:23--  https://codeload.github.com/angusev/coursework_template/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ] 823.42K  --.-KB/s    in 0.05s   

2023-05-10 17:34:23 (15.2 MB/s) - ‘master.zip’ saved [843180]

